In [2]:
# LIBS IMPORT
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [413]:
# READ DATASET
df = pd.read_csv("../../dataset/data_vivareal_21_03_2022_18_38_14.csv").drop(0)

In [414]:
df.head()

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date
1,2553339688,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 597 - Água Verde...",51 m²,2 Quartos,1 Banheiro,1 Vaga,Aceita animais\nCozinha americana\nMobiliado\n...,R$ 325.000,Condomínio: R$ 350,2022-03-21 18:38:38
2,2074895474,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Professor Guido Straube, 775 - Vila Izabel...",72-78 m²,3 Quartos,2 Banheiros,2 Vagas,NaN,R$ 598.000,NaN,2022-03-21 18:38:38
3,2540019474,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 316 - Água Verde...",94 m²,3 Quartos,2 Banheiros,2 Vagas,Mobiliado\nChurrasqueira\nElevador\nCondomínio...,R$ 680.000,Condomínio: R$ 530,2022-03-21 18:38:38
4,2518442439,https://www.vivareal.com.br/imovel/apartamento...,"Rua Coronel Dulcídio, 1756 - Água Verde, Curit...",54 m²,2 Quartos,1 Banheiro,-- Vaga,Mobiliado\nPortão eletrônico\nAceita animais\n...,R$ 215.000,Condomínio: R$ 345,2022-03-21 18:38:39
5,2477728884,https://www.vivareal.com.br/imovel/apartamento...,"Avenida Iguaçu, 2960 - Água Verde, Curitiba - PR",102 m²,3 Quartos,3 Banheiros,2 Vagas,Piscina\nMobiliado\nChurrasqueira\nElevador\nC...,R$ 650.000,Condomínio: R$ 1.000,2022-03-21 18:38:39


In [415]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36617 entries, 1 to 36617
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           36617 non-null  object
 1   url          36617 non-null  object
 2   address      36617 non-null  object
 3   area         36617 non-null  object
 4   rooms        36617 non-null  object
 5   batrooms     36617 non-null  object
 6   garage       36617 non-null  object
 7   amenities    26952 non-null  object
 8   value        36617 non-null  object
 9   condominium  16795 non-null  object
 10  date         36617 non-null  object
dtypes: object(11)
memory usage: 3.4+ MB


# Cleaning data

In [416]:
def cleanData(df, feature, split_character, idx):
    return df[feature].apply(lambda x : x.split(split_character)[idx])

### Area
51m^2 -> 51

In [417]:
# The area column have the m^2 string, remove it and let only the number
# 51m^2 -> 51
df["area"] = cleanData(df, "area", " ", 0)

# EXISTS SOME NEW APARTMENTS WITH area WITH TWO NUMBERS: 72-78, I WILL GET THE SMALL VALUE BECAUSE THE APARTMENT VALUE INIT 
# THE SMALL VALUE
df["area"] = cleanData(df, "area", "-", 0)

### Rooms

In [418]:
# rooms:
# 2 Quantos -> 2
# And remove strange rooms lines
df["rooms"] = cleanData(df, "rooms", " ", 0)
df[["rooms"]].value_counts()

rooms
3        15708
2        11187
1         4364
4         2795
2-3        994
3-4        519
1-3        424
1-2        331
5          102
rooms       69
2-4         61
--          49
6           10
20           2
12           1
19           1
dtype: int64

In [419]:
df["rooms"] = cleanData(df, "rooms", "-", 0)
df = df.loc[~(df["rooms"] == "rooms") & ~(df["rooms"] == "")]
df[["rooms"]].value_counts()

# WE CAN SEE SOME OUTLIERS, A HOUSE WITH 20 ROOMS IS STRANGE. AFTER WE WILL REMOVE ALL THE OUTLIERS

rooms
3        16227
2        12242
1         5119
4         2795
5          102
6           10
20           2
12           1
19           1
dtype: int64

### Batrooms

In [420]:
df["batrooms"] = cleanData(df, "batrooms", " ", 0)
df["batrooms"] = cleanData(df, "batrooms", "-", 0)
df = df.loc[~(df["batrooms"] == "")]

In [421]:
df["batrooms"].value_counts()
# HERE WE HAVE OUTLIER TO

1     12037
2     11956
3      5540
4      3237
5      2541
6       811
7       147
8        97
9        27
10       27
11       16
16        3
20        2
Name: batrooms, dtype: int64

### Garage

In [422]:
df["garage"] = cleanData(df, "garage", " ", 0)
df["garage"] = cleanData(df, "garage", "-", 0)

In [423]:
df["garage"].value_counts()

1     15899
2     10037
       5460
3      3139
4      1609
5       225
6        45
10        6
40        3
8         3
7         3
52        2
9         2
12        1
11        1
14        1
80        1
30        1
18        1
16        1
36        1
Name: garage, dtype: int64

In [424]:
# I WILL ADD 0 TO THE MISSING, AFTER I WILL ADD A VALUE THAT REPRESENT BETTER THIS FEATURES, LIKE MEAN FOR EXAMPLE
# I NOT WANT TO REMOVE THIS LINE BECAUSE EXISTS 5460 ITENS, IT IS A BIG AMOUNT OF INFORMATION
df.loc[df["garage"] == "", ["garage"]] = 0

### Value

In [425]:
# GET THE FIRST ELEMENT OF THE 
value_temp_list = df["value"].apply(lambda x : x.split(" ")[0])

In [426]:
# EXISTS SOM DIFFERENTS FORMATATION VALUES, THE R$, De AND Sob I WILL KEEP, THE OTHERS HAVE A SMALL NUMBER SO I WILL REMOVE THEM
value_temp_list.value_counts()

R$     35504
De       734
Sob      170
A         33
Name: value, dtype: int64

In [427]:
df.loc[value_temp_list == "value"].head(5)

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date


In [428]:
df.loc[value_temp_list == "A"].head(5)

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date
4031,2499477414,https://www.vivareal.com.br/imoveis-lancamento...,"Avenida Anita Garibaldi, 1282 - Cabral, Curiti...",78,3,2,2,NaN,A partir de\nR$ 615.000,NaN,2022-03-21 19:23:48
4701,2545075918,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Doutor Motta Júnior, 2600 - Centro, São Jo...",49,2,1,1,NaN,A partir de\nR$ 198.000,NaN,2022-03-21 19:28:41
5823,2549432009,https://www.vivareal.com.br/imoveis-lancamento...,"Rua João Américo de Oliveira, 922 - Cabral, Cu...",180,3,5,3,NaN,A partir de\nR$ 2.597.770,NaN,2022-03-21 19:36:13
5830,2520668057,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Moyses Marcondes, 53 - Juvevê, Curitiba - PR",105,3,3,2,NaN,A partir de\nR$ 1.030.000,NaN,2022-03-21 19:36:15
10640,2549435659,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Manoel Eufrásio, 1270 - Juvevê, Curitiba - PR",128,3,4,2,NaN,A partir de\nR$ 1.352.418,NaN,2022-03-21 20:08:38


In [429]:
print(df.loc[value_temp_list == "value"].shape[0])
print(df.loc[value_temp_list == "A"].shape[0])

0
33


In [430]:
# REMOVE ROWS WITH "value" AND "A"
df = df.loc[~(value_temp_list == "value")]
df = df.loc[~(value_temp_list == "A")]
print(df.loc[value_temp_list == "value"].shape[0])
print(df.loc[value_temp_list == "A"].shape[0])

0
0


In [431]:
df["value"] = cleanData(df, "value", " ", -1)

In [432]:
# REMOVE ALL LINES WITH THE VALUE Consulta
df = df.loc[~(df["value"] == "Consulta")]

In [433]:
# CHECK IF EXISTS SOME RENT APARTMENT
df.loc[df["value"].apply(lambda x : int(x.replace(".", "")) < 5000)]

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date


In [434]:
df["value"] = df["value"].apply(lambda x : int(x.replace(".", "")))

### Condominium

In [435]:
# ALL NULL SET TO 0
df.loc[df["condominium"].isnull(), "condominium"] = "0"

In [437]:
df["condominium"] = cleanData(df, "condominium", " ", -1)

### District

In [439]:
df["address"]

1        Rua Monsenhor Manoel Vicente, 597 - Água Verde...
2        Rua Professor Guido Straube, 775 - Vila Izabel...
3        Rua Monsenhor Manoel Vicente, 316 - Água Verde...
4        Rua Coronel Dulcídio, 1756 - Água Verde, Curit...
5         Avenida Iguaçu, 2960 - Água Verde, Curitiba - PR
                               ...                        
36613    Rua João Batista Zagonel Passos, 484 - Xaxim, ...
36614    Rua Professor Guido Straube, 775 - Vila Izabel...
36615    Rua João Batista Zagonel Passos, 484 - Xaxim, ...
36616          Rua Santo Bozzi, 770 - Xaxim, Curitiba - PR
36617    Rua Tenente Olderico Gabardo, 311 - Xaxim, Cur...
Name: address, Length: 36238, dtype: object

In [451]:
def test(input: str) -> int:
    return input

In [454]:
test("a")

'a'